In [2]:
! pip3 install neo4j

Directories in use:
home:         /afs/cs.wisc.edu/u/z/z/zzheng/Research/catalog-query/build-catalog/neo4j-community-5.20.0
config:       /afs/cs.wisc.edu/u/z/z/zzheng/Research/catalog-query/build-catalog/neo4j-community-5.20.0/conf
logs:         /afs/cs.wisc.edu/u/z/z/zzheng/Research/catalog-query/build-catalog/neo4j-community-5.20.0/logs
plugins:      /afs/cs.wisc.edu/u/z/z/zzheng/Research/catalog-query/build-catalog/neo4j-community-5.20.0/plugins
import:       /afs/cs.wisc.edu/u/z/z/zzheng/Research/catalog-query/build-catalog/neo4j-community-5.20.0/import
data:         /afs/cs.wisc.edu/u/z/z/zzheng/Research/catalog-query/build-catalog/neo4j-community-5.20.0/data
certificates: /afs/cs.wisc.edu/u/z/z/zzheng/Research/catalog-query/build-catalog/neo4j-community-5.20.0/certificates
licenses:     /afs/cs.wisc.edu/u/z/z/zzheng/Research/catalog-query/build-catalog/neo4j-community-5.20.0/licenses
run:          /afs/cs.wisc.edu/u/z/z/zzheng/Research/catalog-query/build-catalog/neo4j-community

In [10]:
from neo4j import GraphDatabase

# Create a new Driver instance
driver = GraphDatabase.driver(
    "neo4j://localhost:7687", 
    auth=("neo4j", "neo4j"))
driver.verify_connectivity()

driver.close()